In [ ]:
import tensorflow as tf
import math
# from datasets import fishes
from datasets import dataset_factory
from preprocessing import preprocessing_factory
from matplotlib import pyplot as plt
import numpy as np
%matplotlib inline

# take an array of shape (n, height, width) or (n, height, width, channels)
# and visualize each (height, width) thing in a grid of size approx. sqrt(n) by sqrt(n)
def vis_square(data, padsize=1, padval=0):
    data -= data.min()
    data /= data.max()
    
    # force the number of filters to be square
    n = int(np.ceil(np.sqrt(data.shape[0])))
    padding = ((0, n ** 2 - data.shape[0]), (0, padsize), (0, padsize)) + ((0, 0),) * (data.ndim - 3)
    data = np.pad(data, padding, mode='constant', constant_values=(padval, padval))
    
    # tile the filters into an image
    data = data.reshape((n, n) + data.shape[1:]).transpose((0, 2, 1, 3) + tuple(range(4, data.ndim + 1)))
    data = data.reshape((n * data.shape[1], n * data.shape[3]) + data.shape[4:])
    
    plt.imshow(data)

In [ ]:
slim = tf.contrib.slim
ssd_shape = [500, 500]
preprocessing_name = "ssd_inception"

with tf.Graph().as_default():
    dataset = dataset_factory.get_dataset(
                "fishes", "validation", "tmp/fish/dataset")
    num_readers = 5
    batch_size = 4
    provider = slim.dataset_data_provider.DatasetDataProvider(
                        dataset,
                        num_readers=num_readers,
                        common_queue_capacity=20 * batch_size,
                        common_queue_min=10 * batch_size,
                        shuffle=True)
    image, label, bboxes, name = provider.get(['image', 'object/label', 'object/bbox', 'name'])
    image_preprocessing_fn = preprocessing_factory.get_preprocessing(
            preprocessing_name, is_training=True)
    image, label, bboxes = image_preprocessing_fn(image, label, bboxes, ssd_shape)
    images, labels, bboxes = tf.train.batch(
          [image, label, bboxes],
          batch_size=batch_size,
          num_threads=1,
          capacity=2 * batch_size,
          dynamic_pad=True)
    with tf.Session() as sess:
        with slim.queues.QueueRunners(sess):
            sess.run(tf.local_variables_initializer())
            np_images, np_bboxes, labels = sess.run([images, bboxes, labels])


In [ ]:
def vis_imgae_bbox(np_images, np_bboxes):
    colors = plt.cm.hsv(np.linspace(0, 1, 21)).tolist()
    plt.rcParams['figure.figsize'] = (5, 5)
    ind = 0
    height, width = np_images[ind].shape[0:2]

    for ind in range(len(np_images)):
        plt.figure()
        plt.imshow(np_images[ind].astype(np.uint8))
        currentAxis = plt.gca()
        j = 3
        for rect in np_bboxes[ind]:
            [ymin, xmin, ymax, xmax] = rect
            coords = (xmin*width, ymin*height), (xmax-xmin)*width, (ymax-ymin)*height
            currentAxis.add_patch(plt.Rectangle(*coords, fill=False, edgecolor=colors[j], linewidth=2))
            j += 1

# vis_imgae_bbox(np_images, np_bboxes)

In [ ]:
# inception_v3

import os

from nets import inception

slim = tf.contrib.slim
ssd_shape = [450, 450]
preprocessing_name = "ssd_inception"


def get_init_fn():
    """Returns a function run by the chief worker to warm-start the training."""
    checkpoint_exclude_scopes=["InceptionV3/Logits", "InceptionV3/AuxLogits"]
    
    exclusions = [scope.strip() for scope in checkpoint_exclude_scopes]

    variables_to_restore = []
    for var in slim.get_model_variables():
        excluded = False
        for exclusion in exclusions:
            if var.op.name.startswith(exclusion):
                excluded = True
                break
        if not excluded:
            variables_to_restore.append(var)
    
    return slim.assign_from_checkpoint_fn(
        os.path.join(checkpoints_dir, 'inception_v3.ckpt'),
        variables_to_restore)


train_dir = 'tmp/fish/inception_ssd/'
fish_data_dir = 'tmp/fish/dataset/'
checkpoints_dir = 'tmp/fish/my_checkpoints/'
batch_size = 8
with tf.Graph().as_default():
    tf.logging.set_verbosity(tf.logging.INFO)
    
    dataset = dataset_factory.get_dataset(
                "fishes", "validation", "tmp/fish/dataset")
    
    provider = slim.dataset_data_provider.DatasetDataProvider(
                        dataset,
                        num_readers=num_readers,
                        common_queue_capacity=20 * batch_size,
                        common_queue_min=10 * batch_size,
                        shuffle=True)
    image_raw, label, bboxes, name = provider.get(['image', 'object/label', 'object/bbox', 'name'])
    image_preprocessing_fn = preprocessing_factory.get_preprocessing(
            preprocessing_name, is_training=True)
    image, label, bboxes = image_preprocessing_fn(image_raw, label, bboxes, ssd_shape)
    image_raw, images, labels, bboxes = tf.train.batch(
          [image_raw, image, label, bboxes],
          batch_size=batch_size,
          num_threads=1,
          capacity=2 * batch_size,
          dynamic_pad=True)
    
    # Create the model, use the default arg scope to configure the batch norm parameters.
    with slim.arg_scope(inception.inception_v3_arg_scope()):
        _, _end_points = inception.inception_v3_base(images)
        
#     checkpoint_path = tf.train.latest_checkpoint(train_dir)
    checkpoint_path = os.path.join(checkpoints_dir, 'inception_v3.ckpt')
    init_fn = slim.assign_from_checkpoint_fn(checkpoint_path, slim.get_variables_to_restore())   
    with tf.Session() as sess:
        with slim.queues.QueueRunners(sess):
            sess.run(tf.local_variables_initializer())
            init_fn(sess)
            np_end_points, np_images, np_bboxes = sess.run([_end_points, images, bboxes])
            plt.rcParams['figure.figsize'] = (30.0, 15.0)
#             for i in range(batch_size): 
#                 _image = np_images[i, :, :, :]
#                 convnet = np_end_points['Mixed_5c'][i]
#                 vis_square(convnet.transpose(2, 0, 1), padval=0.5)

#                 plt.figure()
#                 plt.imshow(_image.astype(np.uint8))
#                 plt.axis('off')
#                 plt.show()

In [ ]:
width, height = ssd_shape
for i in range(batch_size): 
    _image = np_images[i, :, :, :]
    convnet = np_end_points['Mixed_6e'][i]
    vis_square(convnet.transpose(2, 0, 1), padval=0.5)

    colors = plt.cm.hsv(np.linspace(0, 1, 21)).tolist()
    plt.figure()
    plt.imshow(_image.astype(np.uint8))
    currentAxis = plt.gca()
    j = 3
    for rect in np_bboxes[i]:
        [ymin, xmin, ymax, xmax] = rect
        coords = (xmin*width, ymin*height), (xmax-xmin)*width, (ymax-ymin)*height
        currentAxis.add_patch(plt.Rectangle(*coords, fill=False, edgecolor=colors[j], linewidth=2))
        j += 1
#     plt.axis('off')
    plt.show()

In [ ]:
np_end_points['Mixed_7c'].shape

In [ ]:
tf.trainable_variables()